In [55]:
from datasets import load_dataset
bookcorpus_dataset = load_dataset('bookcorpus')

# 查看数据集中包含的split。常见的split有'train', 'test', 'validation'。
print(bookcorpus_dataset.keys())

# 查看某个split（如'train'）的数据大小
print(len(bookcorpus_dataset['train'])*0.01)

# 查看数据集的一些样例
print(bookcorpus_dataset['train'][0])


Found cached dataset bookcorpus (/home/dtdysh/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f)
100%|██████████| 1/1 [00:17<00:00, 17.63s/it]


dict_keys(['train'])
740042.28
{'text': 'usually , he would be tearing around the living room , playing with his toys .'}


In [130]:
print(len(bookcorpus_dataset['train'])/1000)


74004.228


In [125]:
import uuid
import json
import random
from tqdm import tqdm

def convert_to_csqa(example):
    last_word = example["text"].split()[-1]
    csqa_example = {
        "id": str(uuid.uuid4()),
        "question": {
            "choices": [{"label": "A", "text": last_word}],
            "stem": example["text"]
        },
        "answerKey": "A"
    }
    return csqa_example

dataset_size = 200000

# Generate a list of random indexes
all_indexes = list(range(len(bookcorpus_dataset['train'])))
random_indexes = random.sample(all_indexes, dataset_size)

train_size = int(dataset_size * 0.8)
test_size = int(dataset_size * 0.1)
dev_size = dataset_size - train_size - test_size

ids_train = []

# Write the training data and testing data
with open('../data/bookcorpus/train_rand_split.jsonl', 'w') as f:
    for i in tqdm(range(train_size + test_size), desc="Writing train and test data"):
        csqa_example = convert_to_csqa(bookcorpus_dataset['train'][random_indexes[i]])
        if i < train_size:
            ids_train.append(csqa_example["id"])
        f.write(json.dumps(csqa_example) + '\n')

# Write the dev data
with open('../data/bookcorpus/dev_rand_split.jsonl', 'w') as f:
    for i in tqdm(range(train_size + test_size, dataset_size), desc="Writing dev data"):
        csqa_example = convert_to_csqa(bookcorpus_dataset['train'][random_indexes[i]])
        f.write(json.dumps(csqa_example) + '\n')

# Write IDs for training set
with open('../data/bookcorpus/inhouse_split_qids.txt', 'w') as f:
    for id in ids_train:
        f.write(f"{id}\n")

print("done")


Writing dev data: 100%|██████████| 20000/20000 [00:04<00:00, 4957.55it/s]


done


### Then create or copy a test_rand_split_not_answers.jsonl as test set. But we will only use inhouse test set

In [103]:

import re
BLANK_STR = "___"

def create_hypothesis(fitb: str, choice: str, ans_pos: bool) -> str:

    if ". " + BLANK_STR in fitb or fitb.startswith(BLANK_STR):
        choice = choice[0].upper() + choice[1:]
    else:
        choice = choice.lower()
    # Remove period from the answer choice, if the question doesn't end with the blank
    if not fitb.endswith(BLANK_STR):
        choice = choice.rstrip(".")
    # Some questions already have blanks indicated with 2+ underscores

    choice = choice.replace("\\", "")

    if not ans_pos:
        try:
            hypothesis = re.sub("__+", choice, fitb)
        except:
            print()
            print (choice, fitb)
            print('@@@')
        return hypothesis
    choice = choice.strip()
    m = re.search("__+", fitb)
    start = m.start()

    length = (len(choice) - 1) if fitb.endswith(BLANK_STR) and choice[-1] in ['.', '?', '!'] else len(choice)
    hypothesis = re.sub("__+", choice, fitb)

    return hypothesis, (start, start + length)


fitb = 'my mom couldnt answer anyone as well because she was shocked seen ___ happen and i didnt have a spare second to answer anyone as soon as the nurses came they took me to emergency room for an operation to take out the broken glass pieces inside my skin..'

choice = '\skin'
# print (choice, fitb)

create_hypothesis(fitb=fitb,choice=choice,ans_pos=False)

'my mom couldnt answer anyone as well because she was shocked seen skin happen and i didnt have a spare second to answer anyone as soon as the nurses came they took me to emergency room for an operation to take out the broken glass pieces inside my skin..'

NameError: name 'torch' is not defined